#Initial Settings

In [ ]:
from google.colab import drive 
drive.mount('/content/gdrive/')
path='/content/gdrive/My Drive/Colab Notebooks/06_Anomaly/'

import os
os.chdir(path)

!pip install timm
!pip install ttach

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


#Library

In [ ]:
import albumentations as A
import torch.nn as nn
import ttach as tta
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np 
import random
import torch
import timm
import cv2
import os
import time
import sys

from torch.utils.data import Dataset, DataLoader
from albumentations.pytorch import ToTensor
from glob import glob
from tqdm import tqdm

device = torch.device('cuda')

# 학습을 위한 Hyperparameter 셋팅

In [ ]:
seed = 51
random.seed(seed)
np.random.seed(seed=seed)
torch.manual_seed(seed)

lr = 1e-3
folds = 5
batch_size = 16

resized_image = 512
crop_image = 498

#resized_image = 700
#crop_image = 670

# image augmentation

In [ ]:
albumentations_transform = A.Compose([
    A.RandomCrop(crop_image, crop_image),
    A.HorizontalFlip(p=0.5), # Same with transforms.RandomHorizontalFlip()
    A.VerticalFlip(p=0.5),
    A.Rotate(p=0.5),
    A.GridDistortion(always_apply=False, p=0.5, num_steps=10, distort_limit=(-0.2, 0.2), interpolation=2, border_mode=0),
    A.Cutout(always_apply=False, p=0.5, num_holes=40, max_h_size=10, max_w_size=10),
    ToTensor()
])
albumentations_transform_pred = A.Compose([
    ToTensor()
])

#Custom Dataset & Model

In [ ]:
class Custom_dataset(Dataset):
    def __init__(self, img_paths, labels, mode='train',transform = None):
        self.img_paths = img_paths
        self.labels = labels
        self.mode = mode
        self.transform = transform

    def __len__(self):
        return len(self.img_paths)

    def __getitem__(self, idx):
        img = self.img_paths[idx]
        if self.transform:
            img = self.transform(image=img)['image']

        label = self.labels[idx]
        return img, label

    def get_labels(self):
        return self.labels

class Network(nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        self.model = timm.create_model('efficientnet_b4', num_classes=88)
        
    def forward(self, x):
        x = self.model(x)
        return x

In [ ]:
def img_load(path):
    img = cv2.imread(path)[:,:,::-1]
    img = cv2.resize(img, (resized_image, resized_image))
    img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    return img

In [ ]:
train_y = pd.read_csv("open/train_df.csv")

train_labels = train_y["label"]
label_unique = sorted(np.unique(train_labels))
label_unique = {key:value for key,value in zip(label_unique, range(len(label_unique)))}
label_decoder = {val:key for key, val in label_unique.items()}

test_png = sorted(glob('open/test/*.png'))
test_imgs = [img_load(n) for n in tqdm(test_png)]

# Test
test_dataset = Custom_dataset(np.array(test_imgs), np.array(["tmp"]*len(test_imgs)), mode='test',transform =albumentations_transform_pred )
test_loader = DataLoader(test_dataset, shuffle=False, batch_size=batch_size)

100%|██████████| 2154/2154 [01:20<00:00, 26.91it/s]


# TTA(Test Time Augmentation)


In [ ]:
transforms = tta.Compose(
    [
     tta.HorizontalFlip(),
     tta.Rotate90(angles=[0,90,180]),
     tta.FiveCrops(crop_image, crop_image)
    ]
)

# Predict

In [ ]:
def predict(models, loader):
  f_pred = []
  for batch in (loader):
    x = torch.tensor(batch[0], dtype = torch.float32, device = device)
    
    for i, transformer in enumerate(transforms):
      aug_x = transformer.augment_image(x)
      
      for fold, model in enumerate(models): 
        with torch.no_grad():
          if fold == 0:
            pred = model(aug_x)
          else:
            pred = pred + model(aug_x)

      if i ==0:
        preds = pred/(len(models))
      else:
        preds = preds + pred/(len(models))
    preds = preds/(30)
    f_pred.extend(preds.argmax(1).detach().cpu().numpy().tolist())
  return f_pred

# Inference Ensemble(6 개)
* Augmentation + Efficient Net (Model 5개)
* Augmentation + Efficient Net + Mixup (Model 1개)

In [ ]:
models = []
for i in range(6):
  model = Network().to(device)
  model.load_state_dict(torch.load("./model/"+str(i)+".pt"))
  models.append(model)

f_pred = predict(models, test_loader)

In [ ]:
label_decoder = {val:key for key, val in label_unique.items()}

f_result = [label_decoder[result] for result in f_pred]

In [ ]:
submission = pd.read_csv("open/sample_submission.csv")

submission["label"] = f_result

submission

In [ ]:
submission.to_csv("submission.csv", index = False)